In [ ]:
import numpy as np
import pandas as pd
import imageio
from scipy import misc
from skimage import data
from scipy import ndimage
from enum import Enum

<h1 align="left" style="font-size:20px"><b> General Methods </b></h1>

In [ ]:
class SmoothMode(Enum):
    GAUSSIANBLUR="GaussianBlur"
    DECAY="Decay"
    CLIPWEAK="CipWeak"


In [ ]:
def smooth_img(image, mode = None):
    #mode checking
    if not isisinstance(mode, SmoothMode):
        raise TypeError('mode must be an instance of SmoothMode Enum')
    if mode is None:
        return image
    elif mode == SmoothMode.GAUSSIANBLUR:
        # Gaussian Blurring with width of 3
        return filters.gaussian_filter(image,1/8)
    elif mode == SmoothMode.DECAY:
        # Decay regularization
        decay = 0.98
        return decay * image
    elif mode == SmoothMode.CLIPWEAK:
        # Clip weak pixel regularization
        percentile = 1
        threshold = np.percentile(np.abs(image),percentile)
        im[np.where(np.abs(im) < threshold)] = 0
        return image
    else:
        # print error message
        raise TypeError('Unknown smooth_img parameter. No smooth_img implemented.')
        #print('Unknown smoothing parameter. No smoothing implemented.')
        return image 